## Explore FEC data.
#### Find out how many federal court judges have donated to political campaigns.

In [2]:
from functools import reduce
import glob
import pandas as pd
from pandas_ods_reader import read_ods

#### Combine lists of Art. III and bankruptcy/magistrate judges and get postal code abbreviations for court.
- Fails to pick up anything other than judges in federal district or bankruptcy courts.
- Stripping out suffixes may be problematic as it may match a "Jr." to a "Sr."
- There are entries like "Mrs. Tom Smith" and stripping out "Mrs." will result in false positives.

In [3]:
# A general flag stating whether or not middle names will be used.  "not_middle=True" means middle names
# will not be used to match.
not_middle = True

In [91]:
def clean_art3():
    """
    Reduce a given court name to the postal code abbreviation of the state.
    """
    state_dict = {'Alabama': 'AL',
                  'Alaska': 'AK',
                  'Arizona': 'AZ',
                  'Arkansas': 'AR',
                  'California': 'CA',
                  'Colorado': 'CO',
                  'Connecticut': 'CT',
                  'Delaware': 'DE',
                  'Florida': 'FL',
                  'Georgia': 'GA',
                  'Hawaii': 'HI',
                  'Idaho': 'ID',
                  'Illinois': 'IL',
                  'Indiana': 'IN',
                  'Iowa': 'IA',
                  'Kansas': 'KS',
                  'Kentucky': 'KY',
                  'Louisiana': 'LA',
                  'Maine': 'ME',
                  'Maryland': 'MD',
                  'Massachusetts': 'MA',
                  'Michigan': 'MI',
                  'Minnesota': 'MN',
                  'Mississippi': 'MS',
                  'Missouri': 'MO',
                  'Montana': 'MT',
                  'Nebraska': 'NE',
                  'Nevada': 'NV',
                  'New Hampshire': 'NH',
                  'New Jersey': 'NJ',
                  'New Mexico': 'NM',
                  'New York': 'NY',
                  'North Carolina': 'NC',
                  'North Dakota': 'ND',
                  'Ohio': 'OH',
                  'Oklahoma': 'OK',
                  'Oregon': 'OR',
                  'Pennsylvania': 'PA',
                  'Rhode Island': 'RI',
                  'South Carolina': 'SC',
                  'South Dakota': 'SD',
                  'Tennessee': 'TN',
                  'Texas': 'TX',
                  'Utah': 'UT',
                  'Vermont': 'VT',
                  'Virginia': 'VA',
                  'Washington': 'WA',
                  'West Virginia': 'WV',
                  'Wisconsin': 'WI',
                  'Wyoming': 'WY',}
    
    art3 = pd.read_csv('art_3.csv')
    art3 = art3[['First Name', 'Middle Name', 'Last Name', 'Court Name (1)']]
    
    art3 = art3.rename(columns={'First Name': 'NAME_FIRST',
                                'Middle Name': 'NAME_MIDDLE',
                                'Last Name': 'NAME_LAST',
                                'Court Name (1)': 'COURT'})
    
    pattern = '|'.join(['U.S. District Court for the ', 
                        'Eastern District of ', 
                        'Western District of ',
                        'Southern District of ',
                        'Northern District of ',
                        'Central District of ',
                        'Middle District of ',
                        'District of '])
    
    art3['COURT'] = art3['COURT'].str.replace(pattern, '')
    art3['COURT'] = art3['COURT'].map(state_dict)
    return art3


def clean_bnk_mag():
    """
    Read bankruptcy/magistrate sheet.
    """
    bnk_mag = read_ods('bnk_mag.ods', 'data')
    bnk_mag = bnk_mag[['NAME_FIRST', 'NAME_MIDDLE', 'NAME_LAST', 'COURT']]
    bnk_mag['COURT'] = bnk_mag['COURT'].str.split(',', expand=True)
    return bnk_mag


def combine_judge_files():
    """
    Combine Article III and bankruptcy/magistrate sheets.  Transform courts to postal codes.  
    Add a column comprised of "lastname_firstname_state" to match with FEC data.
    """
    art3 = clean_art3()
    bnk_mag = clean_bnk_mag()
    df = pd.concat([art3, bnk_mag], ignore_index=True)
    df = df.drop_duplicates()
    df = df.applymap(lambda x: x.strip().upper().strip('.') if isinstance(x, str) else x)
    if not_middle:
        df['MATCH_STR'] = df['NAME_LAST'] + "_" + df['NAME_FIRST'] + "_" + df['COURT']
    else:
        df['MATCH_STR'] = df['NAME_LAST'] + "_" + df['NAME_FIRST'] + "_" + df['NAME_MIDDLE'] + "_" + df['COURT']
    # Dropping any judge that did not return a match string (which needs a state, which Circuit and others don't have.)
    df = df.loc[df['MATCH_STR'].notnull()]
    return df

In [92]:
judges = combine_judge_files()

In [93]:
def get_fec_names(file_path):
    """
    Read FEC individual files and test for presence of judge.  Return results that match.
    """

    indiv_vars = ['CMTE_ID', 
                  'AMNDT_IND', 
                  'RPT_TP', 
                  'TRANSACTION_PGI', 
                  'IMAGE_NUM', 
                  'TRANSACTION_TP', 
                  'ENTITY_TP', 
                  'NAME',
                  'CITY', 
                  'STATE', 
                  'ZIP_CODE', 
                  'EMPLOYER', 
                  'OCCUPATION', 
                  'TRANSACTION_DT', 
                  'TRANSACTION_AMT', 
                  'OTHER_ID',
                  'TRAN_ID', 
                  'FILE_NUM', 
                  'MEMO_CD', 
                  'MEMO_TEXT', 
                  'SUB_ID',]
    
    var_types = {'CMTE_ID': str, 
                 'AMNDT_IND': str, 
                 'RPT_TP': str, 
                 'TRANSACTION_PGI': str, 
                 'IMAGE_NUM': str, 
                 'TRANSACTION_TP': str, 
                 'ENTITY_TP': str, 
                 'NAME': str,
                 'CITY': str, 
                 'STATE': str, 
                 'ZIP_CODE': str, 
                 'EMPLOYER': str, 
                 'OCCUPATION': str, 
                 'TRANSACTION_DT': str, 
                 'TRANSACTION_AMT': str, 
                 'OTHER_ID': str, 
                 'TRAN_ID': str, 
                 'FILE_NUM': str, 
                 'MEMO_CD': str, 
                 'MEMO_TEXT': str, 
                 'SUB_ID': str,}

    pattern = '|'.join(['MR.',
                        'MS.',
                        'MRS.',
                        'DR.',
                        'JR',
                        'SR.',
                        'III',])
    
    chunk_list = []

    for df in pd.read_csv(file_path, sep='|', names=indiv_vars, encoding='latin1', dtype=var_types, chunksize=100000):
        df['ALT_NAME'] = df['NAME'].str.upper().str.strip()
        # An example entry:  "Chiles, Earle M."
        if not_middle:
            df['ALT_NAME'] = df['ALT_NAME'].str.replace(', ', '_')
            df['ALT_NAME'] = df['ALT_NAME'].str.split(' ').str[0]
        else:
            df['ALT_NAME'] = df['ALT_NAME'].str.replace(pattern, '')
            df['ALT_NAME'] = df['ALT_NAME'].str.replace(', ', '_')
            df['ALT_NAME'] = df['ALT_NAME'].str.replace(' ', '_')
        df['MATCH_STR'] = df['ALT_NAME'] + '_' + df['STATE'].str.strip() 
        df = df.loc[df['MATCH_STR'].isin(judges['MATCH_STR'])]
        df['FILE'] = file_path
        chunk_list.append(df)

    df_all = pd.concat(chunk_list, ignore_index=True)
    return df_all

In [94]:
def concat_fec_files():
    """
    Concatenate FEC files, subsetting by MATCH_STR as we go along.
    """

    fec_file_list = glob.glob('../data/individual-unzip/*')
    df_list = map(get_fec_names, fec_file_list)
    df_all = pd.concat(df_list, ignore_index=True)
    return df_all

#### Generate files.

In [95]:
fec = concat_fec_files()
# 1981 results (not_middle=False)
# fec.to_csv('first-middle-last-state.csv', index=False)
# 58215 results (not_middle=True)
fec.to_csv('first-last-state.csv', index=False)

### Match individual files from above with candidates.
- If concatenating two files generated above, need to remove duplicates from merged df.
- Committee could support more than one candidate and can work across election cycles.

#### Get a list of CMTE_IDs from individual data files.  We use this to subset the committee-candidates files.

In [7]:
df = pd.read_csv('first-last-state.csv')
# Create set of all CMTE_IDs.
comm_list = df['CMTE_ID'].tolist()
print("List of CMTE_IDs:", len(comm_list))
comm_set = set(comm_list)
print("Set of CMTE_IDs:", len(comm_set))

List of CMTE_IDs: 58215
Set of CMTE_IDs: 4978


#### Generate a subset containing relevant entries from committee-candidate files.

In [18]:
def get_comm_cand(file_path):
    
    comm_cand_vars = ['CAND_ID',
                      'CAND_ELECTION_YR', 
                      'FEC_ELECTION_YR', 
                      'CMTE_ID', 
                      'CMTE_TP', 
                      'CMTE_DSGN', 
                      'LINKAGE_ID', 
                     ]
    
    df = pd.read_csv(file_path, 
                     sep='|',
                     names=comm_cand_vars, 
                     encoding='latin1', 
                    )
    
    df = df.loc[df['CMTE_ID'].isin(comm_set)]
    return df

In [21]:
# Create a df of all relevant committee-candidate linkages.
fec_file_list = glob.glob('../data/cand-comm/*')
df_list = map(get_comm_cand, fec_file_list)
all_comm = pd.concat(df_list, ignore_index=True)
all_comm.to_csv('get-candidate.csv', index=False)

#### Get relevant candidates.

In [22]:
def get_candidate(file_path):
    
    cand_vars = ['CAND_ID', 
                 'CAND_NAME', 
                 'CAND_PTY_AFFILIATION', 
                 'CAND_ELECTION_YR', 
                 'CAND_OFFICE_ST', 
                 'CAND_OFFICE', 
                 'CAND_OFFICE_DISTRICT', 
                 'CAND_ICI', 
                 'CAND_STATUS', 
                 'CAND_PCC', 
                 'CAND_ST1', 
                 'CAND_ST2', 
                 'CAND_CITY', 
                 'CAND_ST', 
                 'CAND_ZIP', 
                ]
    
    df = pd.read_csv(file_path, 
                     sep='|',
                     names=cand_vars, 
                     encoding='latin1', 
                    )
    
    df = df.loc[df['CAND_ID'].isin(all_comm['CAND_ID'])]
    return df

In [23]:
# Create a df of all relevant committee-candidate linkages.
fec_file_list = glob.glob('../data/candidate/*')
df_list = map(get_candidate, fec_file_list)
all_cand = pd.concat(df_list, ignore_index=True)
all_cand.to_csv('all-candidate.csv', index=False)

#### Merge individual, candidate, and candidate-committee files together.

In [67]:
df1 = pd.read_csv('all-candidate.csv')
df1['YEAR'] = df1['CAND_ELECTION_YR'].astype(str)
df1['YEAR'] = df1['YEAR'].str[2:]

df2 = pd.read_csv('get-candidate.csv')
df2['YEAR'] = df2['CAND_ELECTION_YR'].astype(str)
df2['YEAR'] = df2['YEAR'].str[2:]

df3 = pd.read_csv('first-last-state.csv')
df3['YEAR'] = df['FILE'].str.split("/").str[-1]
df3['YEAR'] = df3['YEAR'].str.replace("indiv", "").str.replace(".txt", "")


In [68]:
print("# of obs. in comm-cand file:       ", len(df1))
print("# of obs. in cand file:            ", len(df2))
print("# of obs. in first-last-state file:", len(df3))

# of obs. in comm-cand file:        15298
# of obs. in cand file:             11404
# of obs. in first-last-state file: 58215


In [71]:
merge1 = pd.merge(df3, df2, how='left', on=['CMTE_ID', 'YEAR'])
# comm-cand linkage only works for year 2000 or greater "FEC_ELECTION_YR"
# This still generates duplicates though.
merge1 = merge1[merge1['CAND_ID'].notnull()]
merge1.to_csv('merge1.csv', index=False)

In [72]:
print(merge1)

         CMTE_ID AMNDT_IND RPT_TP TRANSACTION_PGI           IMAGE_NUM  \
0      C00263343         A     YE             NaN         94015111322   
1      C00263343         A     YE             NaN         94015111322   
2      C00263343         A     YE             NaN         94015111322   
3      C00263343         A     YE             NaN         94015111322   
54     C00252601         A     YE             NaN         94015124985   
...          ...       ...    ...             ...                 ...   
80333  C00578013         A    12P           P2016  201607280200325237   
80335  C00608398         A    30R           R2016  201701130200011840   
80336  C00608398         A    30R           R2016  201701130200011948   
80337  C00608398         A    30R           P2016  201703140200080965   
80338  C00608398         A    30R           P2016  201703140200081073   

      TRANSACTION_TP ENTITY_TP                NAME         CITY STATE  ...  \
0                 15       NaN     SCHILLER, 